In [11]:
import causaldag as cd
from causaldag.structure_learning.difference import dci
from random import sample
from causaldag.rand.graphs import unif_away_zero
import itertools as itr
from causaldag.utils.ci_tests import gauss_ci_suffstat
import numpy as np
import random

### Helper functions for generating related but different DAGs

In [12]:
def bounded_change(current_weight):
    if current_weight < 0:
        return random.uniform(.25, 1)
    if current_weight > 0:
        return random.uniform(-1, -.25)

def change_dag(dag, na, nr, nc, rand_change_fn=bounded_change, rand_weight_fn=unif_away_zero):
    na, nr, nc = int(na), int(nr), int(nc)
    removed_arcs = sample(dag.arcs, nr)
    changed_arcs = sample(dag.arcs-set(removed_arcs), nc)
    added_arcs = sample(set(itr.combinations(dag.nodes, 2))-dag.arcs, na)
    
    old_amat = dag.to_amat()
    new_amat = old_amat.copy()
    for i, j in removed_arcs:
        new_amat[i, j] = 0
    for i, j in changed_arcs:
        new_amat[i, j] = rand_change_fn(old_amat[i, j])
    for i, j in added_arcs:
        new_amat[i, j] = rand_weight_fn()
    
    return cd.GaussDAG.from_amat(new_amat)

### Generate two DAGs and samples

In [13]:
random.seed(9348268)

nnodes = 10
nodes = set(range(nnodes))
exp_nbrs = 5
percent_added = .1
percent_removed = .1
percent_changed = .1

d1 = cd.rand.directed_erdos(nnodes, exp_nbrs/(nnodes-1))
g1 = cd.rand.rand_weights(d1)
g2 = change_dag(
    g1, 
    percent_added*(nnodes*(nnodes-1)/2 - d1.num_arcs), 
    percent_removed*d1.num_arcs, 
    percent_changed*d1.num_arcs
)
true_difference = {(i, j) for (i, j), val in np.ndenumerate(g1.to_amat() - g2.to_amat()) if val != 0}

In [14]:
n1 = 10000
n2 = 10000
samples1 = g1.sample(n1)
samples2 = g2.sample(n2)
suff1 = gauss_ci_suffstat(samples1)
suff2 = gauss_ci_suffstat(samples2)

### Run DCI with oracle difference UG

In [15]:
oracle_dug = {
    frozenset({i, j}) for (i, j), val in np.ndenumerate(g1.precision - g2.precision) 
    if val != 0 and i != j
}
print({frozenset({i, j}) for i, j in true_difference} <= oracle_dug)

True


In [20]:
alpha_skeleton = .5
alpha_orient = .01
max_set_size = None

oriented_edges, unoriented_edges = dci(
    nodes, 
    oracle_dug, 
    suff1, 
    suff2, 
    alpha_skeleton=alpha_skeleton, 
    alpha_orient=alpha_orient, 
    max_set_size=max_set_size
)

print("Oriented edges:", oriented_edges)
print("Unoriented edges:", unoriented_edges)
print("True difference:", true_difference)

Oriented edges: {(5, 6), (2, 8), (6, 9), (0, 1)}
Unoriented edges: set()
True difference: {(0, 1), (6, 9), (5, 6), (2, 8), (5, 7), (3, 7)}
